# Coupling modelling
### Please remember to run the installation script and restart Jupyter Lab before using this for the first time!
### Once you did that you can run this program from the "Run" menu, selecting "Run All Cells".
### Finally scroll to the bottom to see (and interact with) the plots.
#### Mariano Curti - v1.0 (09/11/20)

## Fundamentals

For a system containing two chromophores with two energy levels each, the excitonic Hamiltonian is given by:

$$ \hat{H}_{exc} = \begin{pmatrix}
 \epsilon_{i} &V \\
 V &\epsilon_{j}
\end{pmatrix} $$

Where $\epsilon_{i}$ and $\epsilon_{j}$ represent the transition energies for the (isolated) chromophores $i$ and $j$, and $V$ represents the coupling between the excited states.  

For bright and singlet excited states, and interchromophore distances larger than the molecular dimensions, the coupling between chromophores $i$ and $j$ can be obtained from the Point Dipole Approximation:

$$ V = |\mu_{i}||\mu_{j}| \frac{\kappa}{R^3}  \quad \text{with $\kappa=\hat{\mu_{i}} \cdot \hat{\mu_{j}} - 3 (\hat{\mu}_{i} \cdot \hat{R}_{ij}) (\hat{\mu}_{j} \cdot \hat{R}_{ij})$}     $$

Where $R$ is the distance between the chromophore centers and $R_{ij}$ is the vector connecting them.

The diagonalization of the excitonic Hamiltonian yields the energy of the excitonic transitions $E_{1}$ and $E_{2}$ (as the eigenvalues):

$$ \begin{pmatrix}
 \epsilon_{i} &V \\
 V &\epsilon_{j}
\end{pmatrix} \Leftrightarrow \begin{pmatrix}
 E_{1} &0 \\
 0 &E_{2}
\end{pmatrix} $$

For this simple case there is an analytical expression for $E_{1}$ and $E_{2}$:

$$ E_{1,2} = \frac{\epsilon_{i} + \epsilon_{j}}{2} \pm \sqrt{\left(\frac{\epsilon_{i} - \epsilon_{j}}{2}\right)^2 + V^2} $$

From the same diagonalization procedure we get the eigenvectors, which give us the coefficients $C^1_{i},C^1_{j},C^2_{i},C^2_{j}$ from which the excitonic states are built as a linear combination of the localized states (although in this case we don't have an analytical expression for them):

$$ |1\rangle = C^1_{i} |i\rangle + C^1_{j} |j\rangle \quad\quad |2\rangle = C^2_{i} |i\rangle + C^2_{j} |j\rangle$$

To calculate the absorption spectrum of the excitonic system we first calculate the transition dipole moment of each excitonic transition $K$ (i.e. from the ground state to the excitonic state $K$, which can be $1$ or $2$ in our example):

$$ \mu_{K} = C^K_{i} \mu_{i} + C^K_{j} \mu_{j} $$

The extinction coefficient is given then by the following expression, where the sum runs over all excitonic states (2 in this case):

$$ \epsilon(\nu) = 1.089 \times 10^{38} \nu \sum_{k}^M |\mu_{K}|^2 S(\nu - \nu_{K}) $$

Function $S$ represents the homogeneous broadening, and can be simply represented by a Gaussian function:

$$S(\nu)=\frac{1}{\sigma \sqrt{2 \pi}}e^{-\frac{1}{2}\left( \frac{\nu}{\sigma} \right)^2}$$

The circular dichroism spectrum, representing the differential absorption of left-hand and right-hand circularly-polarized light is given by:

$$ \Delta \epsilon (\nu) =  4.355 \times 10^{38} \nu \sum_{k}^M R_{K} S(\nu - \nu_{K}) $$

Where $R_{K}$ is the rotatory strength:

$$ R_{K} = - \frac{\pi \nu_{K}}{2c} \sum_{i,j}^N C^K_{i}C^K_{j} R_{ij} \cdot (\mu_{i} \times \mu_{j})$$

Here the sum runs over all possible chromophore pairs (just $i,j$ here).

# Implementation

The following code is basically a wrapper over a slightly modified version of the EXAT program, developed by the group of Prof. Benedetta Mennucci at Pisa (https://molecolab.dcci.unipi.it/). EXAT provides a simple Python implementation of the above-described equations.

The script presented here allows a simple manipulation of the relative position of two transition dipole moments, to get a quick feedback on how it affects the spectroscopic properties of a hypothetical chromophore dimer. It is a JupyterLab notebook and makes use of the Plotly and iPyWidgets libraries to generate all plots and manipulate the involved variables. It has been tested with JupyterLab 2.1.5; older versions may not work.

Note: the included examples are not meant to provide a quantitative description of each system, but rather a semi-quantitative or qualitative one.

# References

* Curutchet & Mennucci, *Chem. Rev.* 2017, 117, 294−343.
* Jurinovich, Cupellini, Guido, Mennucci, *J. Comp. Chem.* 2018, 39, 279–286.

In [1]:
import plotly.graph_objs as go
from plotly.offline import plot
from plotly.subplots import make_subplots

import ipywidgets
import numpy as np
import util as u
import exat

# Function to rotate vector using spherical coordinates
def rotatevector(v, dtheta_degrees, dphi_degrees):
    dtheta = np.radians(dtheta_degrees)
    dphi = np.radians(dphi_degrees)
    
    r = np.linalg.norm(v)
    phi = np.arctan2(v[1],v[0])
    theta = np.arccos(v[2]/r)
    
    new_theta = theta + dtheta
    new_phi = phi + dphi
    new_x = r * np.sin(new_theta) * np.cos(new_phi)
    new_y = r * np.sin(new_theta) * np.sin(new_phi)
    new_z = r * np.cos(new_theta)
    
    return new_x, new_y, new_z

# Function to calculate PDA coupling, kappa, chromophore distance, and angle between TDs
def coup_k_dist_angle(t0, t1, c0, c1):
    round_digits = 1
    coup, kappa, dist, angle = u.forster(t0, t1, c0, c1)
    
    return int(coup), round(kappa, 2), round(dist, round_digits), round(angle,round_digits)

# Function to get current TDs and chromophore centers from the 3D plot
def get_arrays():
    t0 = np.array([vectors_plot.data[0].x[1]-vectors_plot.data[0].x[0], vectors_plot.data[0].y[1]-vectors_plot.data[0].y[0], vectors_plot.data[0].z[1]-vectors_plot.data[0].z[0]])
    t1 = np.array([vectors_plot.data[1].x[1]-vectors_plot.data[1].x[0], vectors_plot.data[1].y[1]-vectors_plot.data[1].y[0], vectors_plot.data[1].z[1]-vectors_plot.data[1].z[0]])
    c0 = np.array([vectors_plot.data[0].x[0], vectors_plot.data[0].y[0], vectors_plot.data[0].z[0]])
    c1 = np.array([vectors_plot.data[1].x[0], vectors_plot.data[1].y[0], vectors_plot.data[1].z[0]])

    return t0, t1, c0, c1

# Function to calculate angle between the first TDM and the vector connecting the chromophores (not used)
def mu1_R_angle():
    erre=np.array([vectors_plot.data[4].x[1], vectors_plot.data[4].y[1], vectors_plot.data[4].z[1]])
    mu1=np.array([vectors_plot.data[0].x[1], vectors_plot.data[0].y[1], vectors_plot.data[0].z[1]])

    versD1 = erre / np.linalg.norm(erre)
    versD2 = mu1 / np.linalg.norm(mu1)
    dot_product = np.dot(versD1,versD2)
    return np.degrees(np.arccos(dot_product))

# Function to colorize the excitonic coefficients
def colorscale(value):
    if value == 0.0: return "rgb(165,0,38)"
    elif value < 0.1111: return "rgb(215,48,39)"
    elif value < 0.2222: return "rgb(244,109,67)"
    elif value < 0.3333: return "rgb(253,174,97)"
    elif value < 0.4444: return "rgb(254,224,144)"
    elif value < 0.5555: return "rgb(224,243,248)"
    elif value < 0.6666: return "rgb(171,217,233)"
    elif value < 0.7778: return "rgb(116,173,209)"
    elif value < 0.8888: return "rgb(69,117,180)"
    elif value <= 1.0: return "rgb(49,54,149)"

# Range in the 3D plot (in angstrom)
ran = 16    

In [2]:
# Vectors initialization

# Initial positions
Centers = [np.array([0.0, 0.0, 0.0]), np.array([0.0, 0.0, 8.0])]
Dipoles = [np.array([5.0, 0.0, 0.0]), np.array([5.0, 0.0, 0.0])]

vectors = []
colors = ["cornflowerblue","chocolate"]

# Lines representing the transition dipole moments
for i in range(len(Centers)):
    vectors.append(go.Scatter3d(x = [Centers[i][0], Centers[i][0]+Dipoles[i][0]],  y = [Centers[i][1], Centers[i][1]+Dipoles[i][1]], z = [Centers[i][2], Centers[i][2]+Dipoles[i][2]], marker = dict(size = 1, color = colors[i]), line = dict(color = colors[i], width = 6)))

# Dots marking vector heads    
for i in range(len(Centers)):
    vectors.append(go.Scatter3d(x = [Centers[i][0]+Dipoles[i][0]],  y = [Centers[i][1]+Dipoles[i][1]], z = [Centers[i][2]+Dipoles[i][2]], marker = dict(size = 6, color = colors[i])))

# Vector connecting the chromophore centers    
vectors.append(go.Scatter3d(x = [Centers[0][0], Centers[1][0]],  y = [Centers[0][1], Centers[1][1]], z = [Centers[0][2], Centers[1][2]], marker = dict(size = 1, color = colors[i]), line = dict(color = 'gray', width = 6, dash = 'dash')))

data = vectors

layout = go.Layout(
    scene = dict(
        xaxis = dict(nticks=4, range=[-1*ran,ran]),
        yaxis = dict(nticks=4, range=[-1*ran,ran]),
        zaxis = dict(nticks=4, range=[-1*ran,ran])
        ),
    width=700, showlegend=False,
    margin=dict(r=0, l=0, b=0, t=0), scene_aspectmode='cube')

vectors_plot = go.FigureWidget(data=data, layout=layout)

In [3]:
# Widgets initialization

wtheta=ipywidgets.FloatSlider(value=0.0, min=0.0, max=360.0, step=5.0, description='$\\theta$:')
wphi=ipywidgets.FloatSlider(value=0.0, min=0.0, max=360.0, step=5.0, description='$\phi$:')

wx=ipywidgets.FloatSlider(value=0.0, min=-1*ran/2, max=ran/2, step=0.1, description='x:')
wy=ipywidgets.FloatSlider(value=0.0, min=-1*ran/2, max=ran/2, step=0.1, description='y:')
wz=ipywidgets.FloatSlider(value=0.0, min=-1*ran/2, max=ran/2, step=0.1, description='z:')

style = {'description_width': 'initial'}

wsite1=ipywidgets.FloatSlider(value=700.0, min=550.0, max=950.0, step=1.0, description='Site enegy 1 (in nm):', style=style)
wsite2=ipywidgets.FloatSlider(value=700.0, min=550.0, max=950.0, step=1.0, description='Site enegy 2 (in nm):', style=style)

wlabel1=ipywidgets.Label(value="") # This label displays "Distance between centers"
wlabel2=ipywidgets.Label(value="") # This label displays "Angle between vectors"
wlabel3=ipywidgets.Label(value="") # This label displays "Coupling"
wlabel4=ipywidgets.Label(value="$cm^{-1}$") # LaTeX labels are kept separate to avoid flickering.
wlabel5=ipywidgets.Label(value="$\kappa$:")
wlabel6=ipywidgets.Label(value="") # This label displays "Kappa"

# Excitonic coefficients are shown as square buttons
button_size = '60px'
button1 = ipywidgets.Button(description='100%', disabled=True, button_style='', tooltip='', layout=ipywidgets.Layout(width=button_size, height=button_size))
button2 = ipywidgets.Button(description='100%', disabled=True, button_style='', tooltip='', layout=ipywidgets.Layout(width=button_size, height=button_size))
button3 = ipywidgets.Button(description='100%', disabled=True, button_style='', tooltip='', layout=ipywidgets.Layout(width=button_size, height=button_size))
button4 = ipywidgets.Button(description='100%', disabled=True, button_style='', tooltip='', layout=ipywidgets.Layout(width=button_size, height=button_size))

layout = ipywidgets.Layout(width='25%')

buttons = ipywidgets.VBox([ipywidgets.HBox([button1, button2]), ipywidgets.HBox([button3, button4])], layout=layout)

ui = ipywidgets.HBox([ipywidgets.VBox([wtheta, wphi, wx, wy, wz], layout=layout), ipywidgets.VBox([wsite1, wsite2], layout=layout), ipywidgets.VBox([wlabel1, wlabel2, ipywidgets.HBox([wlabel3, wlabel4]), ipywidgets.HBox([wlabel5, wlabel6])], layout=layout), buttons])

wexamples = ipywidgets.Dropdown(options=['', 'H-aggregate', 'J-aggregate', 'High coupling, no CD', 'B800', 'B850', 'Holo 2 cis', 'Holo 2 trans', 'Default'], description='Load Example:', style=style)

# Function to update the labels giving information
def updateLbl(t0, t1, c0, c1, *args):
    coup, kappa, dist, angle = coup_k_dist_angle(t0, t1, c0, c1)
    
    wlabel1.value = "Distance between centers: " + str(dist) + " A"
    wlabel2.value = "Angle between vectors: " + str(angle) + " º"
    wlabel3.value = "Coupling: " + str(coup)
    wlabel6.value = str(kappa)

# Initial calculation of the spectra
t0, t1, c0, c1 = get_arrays()
Site = [1.55, 1.55]
wavenumber,OD,CD,energy,EXCDipo2,EXCRot,coeff = exat.calc_spectra(Site, np.array([t0, t1]), np.array([c0, c1]), 420)
OD_spectrum = go.FigureWidget(data=go.Scatter(x=wavenumber, y=OD, mode='lines'))
CD_spectrum = go.FigureWidget(data=go.Scatter(x=wavenumber, y=CD, mode='lines'))

OD_spectrum.layout.update(margin=dict(r=0, l=0, b=0, t=0), width=800, height=230, yaxis_title="$\epsilon \: / \: mol^{-1} \: cm^{-1}$")
OD_spectrum.update_yaxes(range=[-1000, 80000])
CD_spectrum.layout.update(margin=dict(r=0, l=0, b=0, t=0), width=800, height=230, yaxis_title="$\Delta \epsilon \: / \: mol^{-1} \: cm^{-1}$", xaxis_title="$Wavelength \: / \: nm$")
CD_spectrum.update_yaxes(range=[-50, 50])

# Function to load the different examples (this could be moved to a different file to improve readibility)
def on_change(change):
    print(change)
    if change['type'] == 'change' and change['name'] == 'value':
        if change['new'] == 'H-aggregate':
            wtheta.value = 10.0
            wphi.value = 160.0
            wx.value = -0.6
            wy.value = -0.6
            wz.value = -1.2
            wsite1.value = 700.0
            wsite2.value = 700.0
        elif change['new'] == 'J-aggregate':
            wtheta.value = 10.0
            wphi.value = 160.0
            wx.value = 4.4
            wy.value = 0.0
            wz.value = -3.6
            wsite1.value = 700.0
            wsite2.value = 700.0
        elif change['new'] == 'Default':
            wtheta.value = 0.0
            wphi.value = 0.0
            wx.value = 0.0
            wy.value = 0.0
            wz.value = 0.0
            wsite1.value = 700.0
            wsite2.value = 700.0
        elif change['new'] == 'High coupling, no CD':
            wtheta.value = 00.0
            wphi.value = 180.0
            wx.value = -8.0
            wy.value = 0.0
            wz.value = -8.0
            wsite1.value = 700.0
            wsite2.value = 700.0          
        elif change['new'] == 'B800':
            wtheta.value = 10.0
            wphi.value = 210.0
            wx.value = -6.6
            wy.value = 4.4
            wz.value = -8.0
            wsite1.value = 800.0
            wsite2.value = 800.0             
        elif change['new'] == 'B850':
            wtheta.value = 10.0
            wphi.value = 10.0
            wx.value = -6.8
            wy.value = 4.0
            wz.value = -8.0
            wsite1.value = 850.0
            wsite2.value = 850.0  
        elif change['new'] == 'Holo 2 cis':
            wtheta.value = 10.0
            wphi.value = 20.0
            wx.value = 3.2
            wy.value = 1.8
            wz.value = -4.4
            wsite1.value = 675.0
            wsite2.value = 675.0     
        elif change['new'] == 'Holo 2 trans':
            wtheta.value = 10.0
            wphi.value = 80.0
            wx.value = 8.0
            wy.value = -2.6
            wz.value = 1.4
            wsite1.value = 675.0
            wsite2.value = 675.0  
            
wexamples.observe(on_change)

# Function to update all plots after interacting with the widgets. First part updates the 3D plot, second part updates the spectra and coeff.
def update(theta=wtheta, phi=wphi, x=wx, y=wy, z=wz, label1=wlabel1, label2=wlabel2, label3=wlabel3, site1=wsite1, site2=wsite2):
    with vectors_plot.batch_update():
        new_x, new_y, new_z = rotatevector(Dipoles[1], theta, phi)

        # Vector 1 is always fixed
        
        # Vector 2 is moved according to the widgets' values
        vectors_plot.data[1].x=[x + Centers[1][0], x + Centers[1][0] - new_x]
        vectors_plot.data[1].y=[y + Centers[1][1], y + Centers[1][1] - new_y]
        vectors_plot.data[1].z=[z + Centers[1][2], z + Centers[1][2] - new_z]
        
        # The tip of vector 1 is always fixed
        
        # The tip of vector 2 is moved together with the vector
        vectors_plot.data[3].x = [vectors_plot.data[1].x[1]]
        vectors_plot.data[3].y = [vectors_plot.data[1].y[1]]
        vectors_plot.data[3].z = [vectors_plot.data[1].z[1]]

        # The vector connecting the vector centers is updated
        vectors_plot.data[4].x = [Centers[0][0], x + Centers[1][0]]
        vectors_plot.data[4].y = [Centers[0][1], y + Centers[1][1]]
        vectors_plot.data[4].z = [Centers[0][2], z + Centers[1][2]]

    with vectors_plot.batch_update():
        t0, t1, c0, c1 = get_arrays()
        Site = [1240/site1, 1240/site2]
        t0, t1 = t0*0.4, t1*0.4 # Dipole moment vectors are scaled to give reasonable values in Debye
        wavenumber,OD,CD,energy,EXCDipo2,EXCRot,coeff = exat.calc_spectra(Site, np.array([t0, t1]), np.array([c0, c1]), 200)

        button1.description = str(round(100*coeff[0][0]**2)) + "%"
        button2.description = str(round(100*coeff[0][1]**2)) + "%"
        button3.description = str(round(100*coeff[1][0]**2)) + "%"
        button4.description = str(round(100*coeff[1][1]**2)) + "%"
        
        button1.style.button_color = colorscale(coeff[0][0]**2)
        button2.style.button_color = colorscale(coeff[0][1]**2)
        button3.style.button_color = colorscale(coeff[1][0]**2)
        button4.style.button_color = colorscale(coeff[1][1]**2)
        
        OD_spectrum.data[0].y=OD
        CD_spectrum.data[0].y=CD
        updateLbl(t0, t1, c0, c1)

out = ipywidgets.interactive_output(update, {'theta': wtheta, 'phi': wphi, 'x': wx, 'y': wy, 'z': wz, 'label1': wlabel1, 'label2': wlabel2, 'label3': wlabel3, 'site1': wsite1, 'site2': wsite2})

display(ui, out)
display(wexamples)
        
fig_subplots = ipywidgets.HBox([vectors_plot, ipywidgets.VBox([OD_spectrum, CD_spectrum])])
fig_subplots 

Output()

Dropdown(description='Load Example:', options=('', 'H-aggregate', 'J-aggregate', 'High coupling, no CD', 'B800…

    'data': [{'line': {'color': 'cornflowerblue', 'width': 6},
              'ma…